In [29]:
%reset


In [30]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')


In [31]:
#test_set = pd.read_csv("C:/Users/PC/Desktop/Kandidat - 9. semester/Data Visulization/Respository/Data-Visualisation-project-2022-1/Data/FIFA22_official_data.csv")

#preprocessed = pd.read_csv("C:/Users/PC/Desktop/Kandidat - 9. semester/Data Visulization/Respository/Data-Visualisation-project-2022-1/Data/Preprocessed Data/DF22.csv")

In [32]:
#preprocessed.Position.unique()

In [33]:
def convert_currency(value):
    floatvalue = 0
    strvalue=""
    if type(value) == float:
        floatvalue = str('Unknown')
    elif "K" in value:
        strvalue=value.replace("K","").replace("€","")
        floatvalue=float(float(strvalue))
    elif "M" in value:
        strvalue=value.replace("M","").replace("€","")
        floatvalue=float(float(strvalue)*1000)
    else:
        floatvalue=value.replace("€","")
    return floatvalue

def convert_bodytype(value):
    rvalue = ""
    if type(value) == float:
        rvalue = str("Unknown")
    elif "Lean" in value:
        rvalue = str("Lean")
    elif "Normal" in value:
        rvalue = str("Normal")
    elif "Stocky" in value:
        rvalue = str("Stocky")
    else:
        rvalue = str("Unique")
    return rvalue
    
def weight_converter(value):
    if "kg" in value:
        rvalue = int(value[:-2])
    else:
        rvalue = int(int(value[:-3])*0.4535)
    return rvalue

def height_converter(value):
    if "'" in value:
        rvalue = value.split('\'')
        rvalue = round(int(value[0])*30.48+int(value[2:])*2.54,0)
    if "cm" in value:
        rvalue = value[:-2]
    return rvalue



In [34]:
def preproces_data(data):
    data['Value'] = data['Value'].apply(convert_currency).astype(int)
    data['Wage']  = data['Wage'].apply(convert_currency).astype(int)
    data['Release Clause'] = data['Release Clause'].apply(convert_currency)
    data['Name'] = data['Name'].replace('\d+', '', regex=True).str.lstrip()
    data['Position'] = data['Position'].astype(str).str[-3:].str.replace(">","")
    data['Loaned From'] = data['Loaned From'].str.split('">').str[1].str.replace("</a>","")
    data['Body Type'] = data['Body Type'].apply(convert_bodytype)
    data['Club Logo'] = data['Club Logo'].replace(r'/light_', r'', regex=True)
    data['Height'] = data['Height'].apply(height_converter)
    data['Weight'] = data['Weight'].apply(weight_converter)
    
    # Compact skills

    data['PAC'] = 0 
    data['SHO'] = 0
    data['PAS'] = 0 
    data['DRI'] = 0
    data['DEF'] = 0
    data['PHY'] = 0

    data['PAC'] = round((data['SprintSpeed'] + data['Acceleration'])/2)
    data['SHO'] = round((data['Finishing']+data['Positioning']+data['ShotPower']
                        +data['LongShots']+data['Penalties']+data['Volleys'])/6)
    data['PAS'] = round((data['Vision']+data['Crossing']+data['FKAccuracy']
              +data['LongPassing']+data['ShortPassing']+data['Curve'])/6)
    data['DRI'] = round((data['Agility']+data['Balance']+data['Reactions']
              +data['Composure']+data['BallControl']+data['Dribbling'])/6)
    data['DEF'] = round((data['Interceptions']+data['HeadingAccuracy']+data['DefensiveAwareness']
              +data['StandingTackle']+data['SlidingTackle'])/5)
    data['PHY'] = round((data['Jumping']+data['Stamina']+data['Strength']
              +data['Aggression'])/4)

    data['Club'] = data['Club'].fillna('None')
    data['Work Rate'] = data['Work Rate'].str.replace('N/A/ N/A','Unknown/Unknown')
    data['Real Face'] = data['Real Face'].fillna('No')
    data = data.fillna('Unknown')
    data = data.rename(columns={'Height':'Height(cm)','Weight':'Weight(kg)','Flag':'National Flag',
     'Value': 'Value(1,000€)', 'Wage': 'Wage(1,000€)', 'Work Rate': 'Offense/Defense intensity', 
     'Joined': 'Joined club (date, year)', 'Loaned from': 'Loaned from club'})
    return data

In [35]:
from pathlib import Path
def get_data(paths):
    
    path = Path(str(paths))
    files = Path(path).glob('*.csv') 
    dfs = [pd.read_csv(f) for f in files]

    # Combine the list of dataframes
    df = preproces_data(pd.concat(dfs, ignore_index=True))

    # Add a new column with each year the row is from
    df['Source'] = np.repeat([f'20{i+17}' for i in range(len(dfs))], [len(df) for df in dfs])

    # Making partition of the data - could probably be done in one or two lines
    df[df['Source'] == str(2017)].to_csv(str(paths)+str('/Preprocessed Data/DF17.csv'))
    df[df['Source'] == str(2018)].to_csv(str(paths)+str('/Preprocessed Data/DF18.csv'))
    df[df['Source'] == str(2019)].to_csv(str(paths)+str('/Preprocessed Data/DF19.csv'))
    df[df['Source'] == str(2020)].to_csv(str(paths)+str('/Preprocessed Data/DF20.csv'))
    df[df['Source'] == str(2021)].to_csv(str(paths)+str('/Preprocessed Data/DF21.csv'))
    df[df['Source'] == str(2022)].to_csv(str(paths)+str('/Preprocessed Data/DF22.csv'))
    return df
    

In [36]:
# Insert folder path with the csv files below - make sure to have another folder called
# 'Preprocessed Data' in that folder too (see last lines in the function above)
get_data("C:/Users/PC/Desktop/Kandidat - 9. semester/Data Visulization/Respository/Data-Visualisation-project-2022-1/Data")

,ID,Name,Age,Photo,Nationality,National Flag,Overall,Potential,Club,Club Logo,...,Release Clause,DefensiveAwareness,Kit Number,PAC,SHO,PAS,DRI,DEF,PHY,Source
0,176580,L. Suárez,29,https://cdn.sofifa.com/players/176/580/17_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,92,92,FC Barcelona,https://cdn.sofifa.com/teams/24130.png,...,Unknown,Unknown,Unknown,82.0,89.0,80.0,83.0,Unknown,79.0,2017
1,178518,R. Nainggolan,28,https://cdn.sofifa.com/players/178/518/17_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,86,86,Roma,https://cdn.sofifa.com/teams/5230.png,...,Unknown,Unknown,Unknown,80.0,79.0,76.0,84.0,Unknown,84.0,2017
2,181872,A. Vidal,29,https://cdn.sofifa.com/players/181/872/17_60.png,Chile,https://cdn.sofifa.com/flags/cl.png,87,87,FC Bayern München,https://cdn.sofifa.com/teams/2130.png,...,Unknown,Unknown,Unknown,76.0,81.0,78.0,81.0,Unknown,86.0,2017
3,197445,D. Alaba,24,https://cdn.sofifa.com/players/197/445/17_60.png,Austria,https://cdn.sofifa.com/flags/at.png,86,89,FC Bayern München,https://cdn.sofifa.com/teams/2130.png,...,Unknown,Unknown,Unknown,86.0,76.0,81.0,81.0,Unknown,77.0,2017
4,195864,P. Pogba,23,https://cdn.sofifa.com/players/195/864/17_60.png,France,https://cdn.sofifa.com/flags/fr.png,88,94,Manchester United,https://cdn.sofifa.com/teams/1130.png,...,Unknown,Unknown,Unknown,77.0,82.0,84.0,81.0,Unknown,85.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122007,269526,Deng Xiongtao,19,https://cdn.sofifa.net/players/269/526/23_60.png,China PR,https://cdn.sofifa.net/flags/cn.png,48,61,Meizhou Hakka,https://cdn.sofifa.net/teams/114628/30.png,...,218.0,Unknown,35.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2023
122008,267946,Lim Jun Sub,17,https://cdn.sofifa.net/players/267/946/22_60.png,Korea Republic,https://cdn.sofifa.net/flags/kr.png,48,64,Jeju United FC,https://cdn.sofifa.net/teams/1478/30.png,...,188.0,Unknown,21.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2023
122009,270567,A. Demir,25,https://cdn.sofifa.net/players/270/567/23_60.png,Turkey,https://cdn.sofifa.net/flags/tr.png,51,56,Ümraniyespor,https://cdn.sofifa.net/teams/113796/30.png,...,142.0,Unknown,12.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2023
122010,256624,S. Czajor,18,https://cdn.sofifa.net/players/256/624/21_60.png,Poland,https://cdn.sofifa.net/flags/pl.png,50,65,Fleetwood Town,https://cdn.sofifa.net/teams/112260/30.png,...,214.0,Unknown,40.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2023
